In [1]:
# Loading Titanic dataset and showing the first few coloumns to understand its structure:
import pandas as pd

file_path = 'E:\CODSOFT\Titanic Dataset\Titanic-Dataset.csv'
titanic_data = pd.read_csv(file_path)

titanic_data.head()

<>:4: SyntaxWarning: invalid escape sequence '\C'
<>:4: SyntaxWarning: invalid escape sequence '\C'
C:\Users\Windows 11 Pro\AppData\Local\Temp\ipykernel_8112\4073012979.py:4: SyntaxWarning: invalid escape sequence '\C'
  file_path = 'E:\CODSOFT\Titanic Dataset\Titanic-Dataset.csv'


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# Checking for the missing values:
titanic_data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

## The dataset has missing values in the following columns:
Age: 177 missing values.

Cabin: 687 missing values (a large number).

Embarked: 2 missing values.

# Data Cleaning:

In [7]:
# Imputing missing 'Age' with the median value:
titanic_data['Age'].fillna(titanic_data['Age'].median(), inplace=True)

# Dropping the 'Cabin' column due to many missing values:
titanic_data.drop(columns=['Cabin'], inplace=True)

# Imputing missing 'Embarked' values with the mode (most frequent value):
titanic_data['Embarked'].fillna(titanic_data['Embarked'].mode()[0], inplace=True)

C:\Users\Windows 11 Pro\AppData\Local\Temp\ipykernel_8112\1249840230.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  titanic_data['Age'].fillna(titanic_data['Age'].median(), inplace=True)
C:\Users\Windows 11 Pro\AppData\Local\Temp\ipykernel_8112\1249840230.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting val

In [9]:
# Verifing that there are no more missing values:
titanic_data.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

In [11]:
# Convert 'Sex' and 'Embarked' categorical variables into numerical format using one-hot encoding
titanic_data = pd.get_dummies(titanic_data, columns=['Sex', 'Embarked'], drop_first=True)

# Dropping irrelevant columns 'PassengerId', 'Name', and 'Ticket' for model building
titanic_data.drop(columns=['PassengerId', 'Name', 'Ticket'], inplace=True)

# Check the cleaned and preprocessed data
titanic_data.head()


,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S
0,0,3,22.0,1,0,7.2500,True,False,True
1,1,1,38.0,1,0,71.2833,False,False,False
2,1,3,26.0,0,0,7.9250,False,False,True
3,1,1,35.0,1,0,53.1000,False,False,True
4,0,3,35.0,0,0,8.0500,True,False,True


# Feature Engineering:

In [14]:
#Creating new features 'FamilySize' and 'IsAlone'

# FamilySize is the sum of SibSp (siblings/spouses) and Parch (parents/children) plus the passenger themselves
titanic_data['FamilySize'] = titanic_data['SibSp'] + titanic_data['Parch'] + 1

# IsAlone is a binary feature: 1 if the passenger has no family aboard, 0 otherwise
titanic_data['IsAlone'] = 1  # Initialize to 1 (true)
titanic_data['IsAlone'].loc[titanic_data['FamilySize'] > 1] = 0  # Set to 0 (false) if FamilySize > 1

# Binning 'Age' into categories
titanic_data['AgeBin'] = pd.cut(titanic_data['Age'], bins=[0, 12, 18, 35, 60, 80], labels=['Child', 'Teenager', 'Adult', 'Middle-aged', 'Senior'])

# Binning 'Fare' into categories
titanic_data['FareBin'] = pd.cut(titanic_data['Fare'], bins=[0, 7.91, 14.454, 31, 512], labels=['Low', 'Mid-Low', 'Mid', 'High'])

# Dropping SibSp and Parch as they are replaced by FamilySize and IsAlone
titanic_data.drop(columns=['SibSp', 'Parch'], inplace=True)

# Converting the new 'AgeBin' and 'FareBin' categorical variables into numerical format using one-hot encoding
titanic_data = pd.get_dummies(titanic_data, columns=['AgeBin', 'FareBin'], drop_first=True)

# Checking the updated data with new features
titanic_data.head()

C:\Users\Windows 11 Pro\AppData\Local\Temp\ipykernel_8112\2542573911.py:8: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  titanic_data['IsAlone'].loc[titanic_data['FamilySize'] > 1] = 0  # Set to 0 (false) if FamilySize > 1
C:\Users\Windows 1

,Survived,Pclass,Age,Fare,Sex_male,Embarked_Q,Embarked_S,FamilySize,IsAlone,AgeBin_Teenager,AgeBin_Adult,AgeBin_Middle-aged,AgeBin_Senior,FareBin_Mid-Low,FareBin_Mid,FareBin_High
0,0,3,22.0,7.2500,True,False,True,2,0,False,True,False,False,False,False,False
1,1,1,38.0,71.2833,False,False,False,2,0,False,False,True,False,False,False,True
2,1,3,26.0,7.9250,False,False,True,1,1,False,True,False,False,True,False,False
3,1,1,35.0,53.1000,False,False,True,2,0,False,True,False,False,False,False,True
4,0,3,35.0,8.0500,True,False,True,1,1,False,True,False,False,True,False,False


# Model Building and Evaluation:

In [17]:
from sklearn.model_selection import train_test_split

# Define X (features) and y (target)
X = titanic_data.drop(columns='Survived')
y = titanic_data['Survived']

# Split the data into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Create a Logistic Regression model
model = LogisticRegression(max_iter=1000)

# Train the model
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [21]:
# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model performance
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.8100558659217877
Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.86      0.84       105
           1       0.79      0.74      0.76        74

    accuracy                           0.81       179
   macro avg       0.81      0.80      0.80       179
weighted avg       0.81      0.81      0.81       179



In [23]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for Logistic Regression
param_grid = {
    'C': [0.1, 1, 10, 100],
    'penalty': ['l2']
}

# Perform grid search with cross-validation
grid_search = GridSearchCV(LogisticRegression(max_iter=1000), param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Best parameters
print("Best parameters:", grid_search.best_params_)

# Best model performance on test data
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test)
print("Best Model Accuracy:", accuracy_score(y_test, y_pred_best))


Best parameters: {'C': 10, 'penalty': 'l2'}
Best Model Accuracy: 0.8100558659217877
